In [25]:
import os
import pandas as pd
import sqlite3

In [27]:
db_path = 'sql/eleicao.db' 
connection = sqlite3.connect(db_path)

In [91]:
querry = """
SELECT * 
FROM municipio_nome
"""

municipios = pd.read_sql_query(querry,connection)
municipios.head()

,CD_MUN,NM_MUN
0,2907806,CÍCERO DANTAS
1,2923902,PAU BRASIL
2,2923209,OLIVEIRA DOS BREJINHOS
3,2921500,MONTE SANTO
4,2921203,MIGUEL CALMON


In [207]:
def get_dataset(text):
    text = [i for i in text if i != "\""]
    text = "".join(text)
    
    lines = text.split("\n")
    columns = lines[0].split(";")

    df = []    
    for line in lines[1:]:
        features = line.split(";")
        if len(features) == len(columns):
            df.append(features)
    df = pd.DataFrame(df,columns=columns)

    
    for i in df.columns:
        try:
            column_values = df[i]
            tipo = int
            if "," in list(df[i].values[0]):
                column_values = column_values.apply(lambda x: x.replace(",","."))
                tipo = float
                
            df[i] = column_values.astype(tipo)
        except Exception as e:
            df.drop(i,axis=1,inplace=True)
            
    return df

In [215]:
def clean_municipios(df):
    municipios_num = {int(x[0]):x[1] for x in municipios.values}

    df_filter = df[df['CD_MUN'].apply(lambda x: x in municipios_num.keys())].reset_index(drop=True)

    return df_filter

In [235]:
csv_files = ['Agregados_por_municipios_alfabetizacao_BR.csv',
 'Agregados_por_municipios_basico_BR.csv',
 'Agregados_por_municipios_caracteristicas_domicilio1_BR.csv',
 'Agregados_por_municipios_caracteristicas_domicilio2_BR.csv',
 'Agregados_por_municipios_caracteristicas_domicilio3_BR.csv',
 'Agregados_por_municipios_cor_ou_raca_BR.csv',
 'Agregados_por_municipios_demografia_BR.csv',
 'Agregados_por_municipios_domicilios_indigenas_BR.csv',
 'Agregados_por_municipios_domicilios_quilombolas_BR.csv',
 'Agregados_por_municipios_obitos_BR.csv',
 'Agregados_por_municipios_parentesco_BR.csv',
 'Agregados_por_municipios_pessoas_indigenas_BR.csv',
 'Agregados_por_municipios_pessoas_quilombolas_BR.csv']
csv_names = ['alfabetizacao','basico','domicilio1','domicilio2','domicilio3','cor_raca','demografia',
             'domicilios_indigenas','domicilios_quilombolas','obitos','parentesco','pessoas_indigenas','pessoas_quilombolas']

In [237]:
for file,name in zip(csv_files,csv_names):
    with open("datasets/IBGE/censo/{}".format(file),"r") as file:
        text = file.read()

    df = get_dataset(text)
    df = clean_municipios(df)

    # Converter o DataFrame em uma tabela SQL
    df.to_sql(name, connection, if_exists='replace', index=False)

In [239]:
# Fechar a conexão
connection.close()